In [0]:
import sys, subprocess
color = dbutils.widgets.get('color')
if color == 'silver':
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'databricks-labs-dqx==0.6.0'], check=True)
    dbutils.library.restartPython()


In [0]:
import json
from pathlib import Path
from functions.utility import get_function, create_bad_records_table, apply_job_type, schema_exists, print_settings
from functions.quality import create_dqx_bad_records_table
from functions.history import build_and_merge_file_history
import os

# Variables
color                       = dbutils.widgets.get('color')
job_settings                = json.loads(dbutils.widgets.get('job_settings'))
table                       = job_settings['table']
settings                    = json.loads(next(Path().glob(f'./layer_*_{color}/{table}.json')).read_text())
settings                    = apply_job_type(settings)
dst_table_name              = settings.get("dst_table_name", None)

# Print job and table settings
print_settings(job_settings, settings, color, table)

# One function for pipeline
if 'pipeline_function' in settings:
    pipeline_function = get_function(settings['pipeline_function'])
    pipeline_function(settings, spark)

# Individual functions for each step
elif all(k in settings for k in ['read_function', 'transform_function', 'write_function']):
    read_function = get_function(settings['read_function'])
    transform_function = get_function(settings['transform_function'])
    write_function = get_function(settings['write_function'])

    # Read and transform
    df = read_function(settings, spark)
    df = transform_function(df, settings, spark)

    # Create a DQX bad records table if any checks fail
    if color == 'silver':
        df = create_dqx_bad_records_table(df, settings, spark)

    # Write
    write_function(df, settings, spark)
else:
    raise Exception(f'Could not find any ingest function name in settings.')

# Create a bad records table if bad records files found
if color == 'bronze':
    create_bad_records_table(settings, spark)
